In [1]:
import sqlite3
import pandas as pd
import numpy as np
from API_Key import key, username, password
import requests
import time
import psycopg2 as pg2

In [2]:
def inputting_to_db(username, password, query):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute(query)
    
    conn.commit()
    conn.close()

In [3]:
# Adding data to tables created 
def adding_to_db(username, password, query, data):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.executemany(query, data)
    
    conn.commit()
    conn.close()

# Creating tables based on Riot Games API Structure

In [4]:
query1 = ('''CREATE TABLE summoner(

                leagueId VARCHAR,
                queueType VARCHAR,
                tier TEXT,
                rank TEXT,
                summonerId VARCHAR,
                summonerName TEXT,
                leaguepoints INTEGER,
                wins INTEGER,
                losses INTEGER,
                veteran BOOLEAN,
                inactive BOOLEAN,
                freshBlood BOOLEAN,
                hotStreak BOOLEAN) ''')

In [5]:
query2 = ('''CREATE TABLE userids(

                id TEXT,
                accountId TEXT,
                puuid TEXT PRIMARY KEY,
                name TEXT,
                profileIconId INTEGER,
                revisionDate BIG INT,
                summonerLevel INTEGER) ''')

In [6]:
query3 = ("""CREATE TABLE matchids(
             puuid text PRIMARY KEY,
             match text)""")

In [7]:
inputting_to_db(username, password, query1)

In [8]:
inputting_to_db(username, password, query2)

In [9]:
inputting_to_db(username, password, query3)

# Setting up loop to obtain all ranked TFT players in Diamond and below

In [10]:
def player_collector(key):
    leagues = ['IRON', 'BRONZE', 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
    divisions = ['I', 'II', 'III', 'IV']
    summoner = []
    for league in leagues:
        for division in divisions:
            response = requests.get(
            "https://na1.api.riotgames.com/tft/league/v1/entries/{}/{}?api_key={}".format(league, division, key))
            for user in response.json():
                summoner.append(tuple(user.values()))
    return summoner

In [11]:
players = player_collector(key)

In [12]:
len(players)

4738

# Updating summoner table 

In [13]:
query4 = 'INSERT INTO summoner VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'

In [14]:
adding_to_db(username, password, query4, players)

In [10]:
query5 = 'INSERT INTO userids VALUES (%s,%s,%s,%s,%s,%s,%s)'

In [5]:
def puuid_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT summonerName FROM summoner")
    players = c.fetchall()
    # Getting a list of all summonernames to loop through
    users = [player[0] for player in players]
    summoner = []
    for player in users:
        response = requests.get(
        "https://na1.api.riotgames.com/tft/summoner/v1/summoners/by-name/{}?api_key={}".format(player, key))
        if response.status_code == 200:
            summoner.append(tuple(response.json().values()))
        elif response.status_code == 429:
            print(len(summoner))
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return summoner;

In [6]:
puuid = puuid_collector(key, username, password)

100
rate limit exceeded
waiting 2 minutes
200
rate limit exceeded
waiting 2 minutes
300
rate limit exceeded
waiting 2 minutes
400
rate limit exceeded
waiting 2 minutes
500
rate limit exceeded
waiting 2 minutes
600
rate limit exceeded
waiting 2 minutes
699
rate limit exceeded
waiting 2 minutes
799
rate limit exceeded
waiting 2 minutes
899
rate limit exceeded
waiting 2 minutes
999
rate limit exceeded
waiting 2 minutes
1099
rate limit exceeded
waiting 2 minutes
1199
rate limit exceeded
waiting 2 minutes
1299
rate limit exceeded
waiting 2 minutes
1399
rate limit exceeded
waiting 2 minutes
1499
rate limit exceeded
waiting 2 minutes
1599
rate limit exceeded
waiting 2 minutes
1699
rate limit exceeded
waiting 2 minutes
1799
rate limit exceeded
waiting 2 minutes
1898
rate limit exceeded
waiting 2 minutes
1998
rate limit exceeded
waiting 2 minutes
2097
rate limit exceeded
waiting 2 minutes
2197
rate limit exceeded
waiting 2 minutes
2297
rate limit exceeded
waiting 2 minutes
2397
rate limit excee

In [13]:
adding_to_db(username, password, query5, puuid)

In [36]:
def matchid_collector(key, username, password):
    conn = pg2.connect(database='TFT', user=username, password=password)
    c = conn.cursor()
    c.execute("SELECT puuid FROM userids")
    players = c.fetchall()
    # Getting a list of all puuids to loop through
    users = [player[0] for player in players]
    games = []
    for player in users:
        response = requests.get(
            "https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/{}/ids?count=30&api_key={}".format(player, key))
        if response.status_code == 200:
            for matches in response.json():
                games.append((player, matches))
        elif response.status_code == 429:
            print(len(games), games[-2:])
            print('rate limit exceeded')
            print("waiting 2 minutes")
            time.sleep(120)
        else:
            pass
    return games;

In [37]:
match_histories = matchid_collector(key, username, password)

2803 [('yLC44_Jkngmkw2vL8PLtSx83hFwXSJIKrB-xnftl86c8hyI0fY3Tc7Y1j9oc3iGQcsSPB1Njf3slYQ', 'NA1_3742278015'), ('yLC44_Jkngmkw2vL8PLtSx83hFwXSJIKrB-xnftl86c8hyI0fY3Tc7Y1j9oc3iGQcsSPB1Njf3slYQ', 'NA1_3742218814')]
rate limit exceeded
waiting 2 minutes
5531 [('eL4CeoiAa3lsAkERVsE_7Yyh4m5VIIsBgz0bR8HYol-ry-zDYYwNRKCqZ0JKnBiOlnImd2UXfcEXDw', 'NA1_3739730836'), ('eL4CeoiAa3lsAkERVsE_7Yyh4m5VIIsBgz0bR8HYol-ry-zDYYwNRKCqZ0JKnBiOlnImd2UXfcEXDw', 'NA1_3739607226')]
rate limit exceeded
waiting 2 minutes
8031 [('Q5n6W4N91n3Hny7qEtfvagf8X9lrsTfwCXSRTZZvQnQruuc3mGqPd4362V-qCqXwfhE2o74bXs-jdg', 'NA1_3548379652'), ('Q5n6W4N91n3Hny7qEtfvagf8X9lrsTfwCXSRTZZvQnQruuc3mGqPd4362V-qCqXwfhE2o74bXs-jdg', 'NA1_3548391531')]
rate limit exceeded
waiting 2 minutes
10665 [('ppuEFRdv26V1-YVxKx8PTZ3u-57eSRRzTBlfWkkIYmqZrp1C8kqtQysql70tGliCOVqYXstDY695Vw', 'NA1_3534814320'), ('ppuEFRdv26V1-YVxKx8PTZ3u-57eSRRzTBlfWkkIYmqZrp1C8kqtQysql70tGliCOVqYXstDY695Vw', 'NA1_3474632347')]
rate limit exceeded
waiting 2 minutes
13163 [

94699 [('ibvTY14o-aT29D3f79Tj5KNOkuByCicCO9PVgaCuyFJSX6PKxQbt1_7JtJwkdch7ceh2eW33bJLzug', 'NA1_3751134252'), ('ibvTY14o-aT29D3f79Tj5KNOkuByCicCO9PVgaCuyFJSX6PKxQbt1_7JtJwkdch7ceh2eW33bJLzug', 'NA1_3750938943')]
rate limit exceeded
waiting 2 minutes
97639 [('4p6TqcT1bdtAtZt-oCfIv4k4i5tWUycQSfgxu4VYNtBZMyG6NHug3TxGHlKy6AqgD9zltibICFwJ0A', 'NA1_3723360291'), ('4p6TqcT1bdtAtZt-oCfIv4k4i5tWUycQSfgxu4VYNtBZMyG6NHug3TxGHlKy6AqgD9zltibICFwJ0A', 'NA1_3722894736')]
rate limit exceeded
waiting 2 minutes
100489 [('gUZ-xVUZ07flRdmXeSc7nrbDGZKEcznr8RE80cGu9YwALR6UOJ2_74Qu3n5Wu9FVHi_UvVsCWO1rXQ', 'NA1_3733597734'), ('gUZ-xVUZ07flRdmXeSc7nrbDGZKEcznr8RE80cGu9YwALR6UOJ2_74Qu3n5Wu9FVHi_UvVsCWO1rXQ', 'NA1_3733613101')]
rate limit exceeded
waiting 2 minutes
103369 [('f6wqdLCUs_FpEutGlsj0vPTeOqxaGJsjZVf-Ib5fN1VRmGlGb8MKWVy9mDRrg4glPwfELn7q49lfHA', 'NA1_3756651500'), ('f6wqdLCUs_FpEutGlsj0vPTeOqxaGJsjZVf-Ib5fN1VRmGlGb8MKWVy9mDRrg4glPwfELn7q49lfHA', 'NA1_3756429126')]
rate limit exceeded
waiting 2 minutes
10

In [38]:
query6 = 'INSERT INTO matchids VALUES (%s,%s)'

In [41]:
adding_to_db(username, password, query6, match_histories)